In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch

# Define the paths
trained_model_path = "/opt/notebooks/Chatbot-Credit-Card/backend/models/llama-3.1-8b-Instruct-CC/instruct/checkpoint-52"
# base_model_path = "meta-llama/Llama-3.1-8B"
base_model_path = "/opt/notebooks/Chatbot-Credit-Card/backend/models/llama-3.1-8b-Instruct/"

# BitsAndBytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the tokenizer from the trained model path
tokenizer = AutoTokenizer.from_pretrained(trained_model_path, trust_remote_code=True)

# Do not add special tokens or resize embeddings here

# Load the base model
print("Loading the base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Load PEFT configuration
print("Loading the PEFT configuration...")
peft_config = PeftConfig.from_pretrained(trained_model_path)

# Load the PEFT model
print("Loading the PEFT model...")
model = PeftModel.from_pretrained(
    base_model,
    trained_model_path,
    device_map="auto",
    torch_dtype=torch.float16,
)

print("Model loaded successfully.")

# Verify that the tokenizer and model's embeddings are aligned
print(f"Tokenizer vocabulary size: {len(tokenizer)}")
print(f"Model's embedding size: {model.get_input_embeddings().weight.shape[0]}")


Loading the base model...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading the PEFT configuration...
Loading the PEFT model...
Model loaded successfully.
Tokenizer vocabulary size: 128256
Model's embedding size: 128256


In [2]:
# Prepare the prompt
messages = [
    {"role": "user", "content": "Label Gender: Male. Age: 32 years old years old. Car Ownership: Yes. Property Ownership: Yes. Number of Children: 0. Annual Income: 427500.0. Income Category: Working. Education Level: Higher education. Marital Status: Civil marriage. Housing Type: Rented apartment. Employment Duration: 12.44 years. Mobile Phone: Yes. Work Phone: Yes. Email: No. Family Size: 2.0"}
]

def format_chat_messages(messages):
    formatted_prompt = ""
    for message in messages:
        if message["role"] == "user":
            formatted_prompt += "<start_input>\n"
            formatted_prompt += f"{message['content']}\n"
            formatted_prompt += "<end_input>\n"
    formatted_prompt += "<start_label>\n"  # Indicate that assistant's response should follow
    return formatted_prompt

prompt = format_chat_messages(messages)

In [3]:
# Generate response from the model
print("Generating response...")
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Ensure the input is moved to the correct device (e.g., GPU)

# Generate tokens with the model
outputs = model.generate(
    **inputs,
    max_new_tokens=150,  # Adjust based on how long you want the output
    do_sample=True,      # Enable sampling for more varied responses
    temperature=0.7,     # Adjust for response diversity (lower for more deterministic output)
    top_k=50,            # Consider the top-k tokens for sampling
)

# Decode the generated tokens to text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the response
print("Response:")
print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generating response...


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Response:
<start_input>
Label Gender: Male. Age: 32 years old years old. Car Ownership: Yes. Property Ownership: Yes. Number of Children: 0. Annual Income: 427500.0. Income Category: Working. Education Level: Higher education. Marital Status: Civil marriage. Housing Type: Rented apartment. Employment Duration: 12.44 years. Mobile Phone: Yes. Work Phone: Yes. Email: No. Family Size: 2.0
<end_input>
<start_label>
car_ownership
property_ownership
number_of_children
annual_income
income_category
education_level
marital_status
housing_type
employment_duration
mobile_phone
work_phone
email
family_size
<end_label>
<start_category>
Age: 32 years old
Car Ownership: yes
Property Ownership: yes
Number of Children: 0
Annual Income: 427500
Income Category: Working
Education Level: Higher education
Marital Status: Civil marriage
Housing Type: Rented apartment
Employment Duration: 12.44 years
Mobile Phone: yes
Work Phone: yes
Email: no
Family Size: 2
<end_category>
<start_number>
<car_ownership>
